<a href="https://colab.research.google.com/github/dominique-nshimyimana/Car-State/blob/master/series_car_state_action_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Car State-Action Prection 
#Colab: TensorFlow and GoogleDrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%tensorflow_version 2.x

In [3]:
from google.colab import drive
mounted = '/drive'
drive.mount(mounted)

Mounted at /drive


# NN state Evaluation

In [0]:
import tensorflow as tf
import numpy as np
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


class NeuralDynamics(tf.keras.Model):

    def __init__(self, state_size, action_size):

        super(NeuralDynamics, self).__init__()

        self.training_counter = 0

        if type(state_size) is tuple:
          self.state_size = state_size
          self.action_size = action_size
          self.out_size = state_size[1]
        else:
          self.state_size = state_size
          self.action_size = action_size
          self.out_size = state_size

        self.layer1 = tf.keras.layers.Dense(300, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.1))
        self.dropout = tf.keras.layers.Dropout(rate=0.5)
        self.layer2 = tf.keras.layers.Dense(300, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.1))

        self.layer_state1 = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.1))
        self.layer_state2 = tf.keras.layers.Dense(self.out_size, kernel_regularizer=tf.keras.regularizers.l1(0.1))

        self.known_inputs = False
        self.training_step = 0

    @tf.function
    def call(self, inputs):

        state, action = inputs
        if type(self.state_size) is tuple and self.state_size[0] != state.shape[0]:
          missing_zeros = self.state_size[0]-state.shape[0]
          paddings = tf.constant([[missing_zeros, 0], [0, 0]])
          state = tf.pad(state, paddings, "CONSTANT")

        x = tf.concat((state, action), axis=1)
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.dropout(x)

        x = self.layer_state1(x)
        x = self.dropout(x)
        state = self.layer_state2(x)
        return state

# Prepare Data

In [0]:
# Load dat from folder
# CSV form: Dataset
# Timestamp, x, y, yaw, xvel, yvel, omega, accel, brake, steering
def load_states_actions(data_dir, cutoff_beginning = 300, cutoff_end = 1000, norm=True):
  data_files = [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f)) and ".directory" not in f]

  xs_states = []
  xs_actions = []
  ys = []

  for f in data_files:
      print(f)
      data = np.loadtxt(f, delimiter=', ', skiprows=1, dtype=np.float32)[cutoff_beginning:-cutoff_end, :]
      x_states = [data[i, 1:-3] for i in range(len(data))]
      x_actions = [data[i, -3:] for i in range(len(data))]
      y = x_states[1:]
      x_states, x_actions = x_states[:-1], x_actions[:-1]
      xs_states += x_states
      xs_actions += x_actions

  xs_states = np.vstack(xs_states)
  xs_actions = np.vstack(xs_actions)

  scaler = MinMaxScaler()
  xs_states = scaler.fit_transform(xs_states)
  xs_actions = scaler.fit_transform(xs_actions)
  return xs_states, xs_actions

# convert an array of values into a dataset matrix
# State(s) at a given time t (or time series until t) and Y is the state at the next time (t + 1).
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		dataX.append(np.squeeze(dataset[i:(i+look_back)])) #dataset[i:(i+look_back), 0]
		dataY.append(dataset[i + look_back]) #dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

def takeSlice(arr, fr, to, name):
    
    result = arr[:,fr:to,:]
    print(name + ": start at " + str(fr) + " - shape: " + str(result.shape))
    return result

### Load Data

In [8]:
# Load Data
data_dir = "/drive/My Drive/neuronyte_logging"
xs_states, xs_actions = load_states_actions(data_dir=data_dir)

/drive/My Drive/neuronyte_logging/NeuroNyte_1585748620.316643
/drive/My Drive/neuronyte_logging/NeuroNyte_1585745385.311904


### Preparre and Look at Data ...

In [9]:
# Split data
x_states_train, x_states_test, x_actions_train, x_actions_test = train_test_split(xs_states, xs_actions, test_size=0.10)

# How many steps/state in back from t and # How many steps/state in future/forward from t
look_back = 10
look_forward = 1

# reshape into X=t and Y=t+1 for states
x_states_train, y_train = create_dataset(x_states_train, look_back)
x_states_test, y_test = create_dataset(x_states_test, look_back)

# Visualize X=t and Y=t+1
t = 3
print(y_test[t])
print(x_states_test[t+1])

XS_shape, XA_shape, y_shape = x_states_train.shape, x_actions_train.shape, y_train.shape
X = np.array(x_states_train).reshape(XS_shape[0], look_back, XS_shape[-1])
Y = np.array(y_train).reshape(y_shape[0], look_forward, y_shape[-1])

[0.98198146 0.66104215 0.5010233  0.50080323 0.9988941  0.49995062]
[[0.67504954 0.5637433  0.1148226  0.33502385 0.32550812 0.5007954 ]
 [0.18454403 0.14716876 0.7983072  0.67529637 0.43282956 0.4987799 ]
 [0.68092257 0.11942458 0.6775433  0.63326335 0.5805489  0.5007576 ]
 [0.76345164 0.07020801 0.2667284  0.01600128 0.52763337 0.49998266]
 [0.12828135 0.11987418 0.9549924  0.57600474 0.1490672  0.50053215]
 [0.2938729  0.5442023  0.077445   0.33114904 0.23117295 0.50036293]
 [0.9628264  0.40464428 0.5218235  0.56318164 0.98012936 0.499947  ]
 [0.65710247 0.52185863 0.85714054 0.5062765  0.2780962  0.5013704 ]
 [0.45695096 0.14309996 0.22922507 0.3351694  0.478396   0.49997175]
 [0.98198146 0.66104215 0.5010233  0.50080323 0.9988941  0.49995062]]


# Initialize Model

In [0]:
decay=tf.keras.optimizers.schedules.ExponentialDecay(0.0004, 20000, 0.99)
optimizer = tf.keras.optimizers.Adam(learning_rate=decay)
model = NeuralDynamics((24, 6), 2)
EPOCHS = 2
batchsize = 2000

# Training

In [0]:
print(x_states_train.shape, x_actions_train.shape, y_train.shape)
model.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())

# checkpoint
'''
filepath=join(mounted, "My Drive/NeuralModel/lstm_car_state/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5")
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# Fit the model
history = model.fit(X, Y, epochs=EPOCHS, validation_split=0.2, verbose=1, batch_size=20000, callbacks=callbacks_list)
'''

history = model.fit(X, Y, epochs=EPOCHS, validation_split=0.2, verbose=1, batch_size=batchsize)
model.save(join(mounted, 'My Drive/NeuralModel/lstm_car_state/lstm_fin.h5'))

# Testing

In [0]:
 xs_shape, xa_shape, yt_shape = x_states_test.shape, x_actions_test.shape, y_test.shape
X = np.array(x_states_test).reshape(xs_shape[0], look_back, xs_shape[-1])
Y = np.array(y_test).reshape(yt_shape[0], look_forward, yt_shape[-1])

print('Test(xs, sa, y): ', x_states_test.shape, x_actions_test.shape, y_test.shape)
result = model.predict((x_states_test, x_actions_test))
print('Result: ', result.shape)

# Visualization of result

In [0]:
xs_states_inpt = np.squeeze(x_states_test[:,:,0:2], axis=0)
xs_states_pred = np.squeeze(result[:,:,0:2], axis=0)

fig,ax = plt.subplots(1,1,figsize=(16,16))
ax.plot(*zip(*xs_states_inpt), 'r')
ax.plot(*zip(*xs_states_pred),'b')
plt.xlabel('x pos')
plt.ylabel('y pos')
plt.suptitle("yy pasition as one feature")
plt.show()